In [ ]:
import os
import pandas as pd

In [2]:
import re

In [3]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [4]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [ ]:
df = pd.read_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]001_dataframe_full.p"))

In [ ]:
df.head()

In [6]:
df_manual_labels = pd.read_csv(os.path.join(cfg.get("directory", "dataset"), cfg.get("dataset", "wikidata_aux"), cfg.get("dataset", "wikidata_manual_labels")), engine='python')
df_manual_labels

,match,label
0,undo,REVERT
1,undoing,REVERT
2,undid,REVERT
3,rv,REVERT
4,revert,REVERT
5,reverted,REVERT
6,reverting,REVERT
7,rb,REVERT
8,rollback,REVERT
9,rolled back,REVERT


In [9]:
comment_regex = re.compile(r"\/\* (.*):(\d)\|(.*) \*\/")

def split_comment(comment):
    if comment == "" or pd.isna(comment):
        base_action = ""
        digit = ""
        language_code = ""
    else:
        match = re.match(comment_regex, comment)
        if match is not None:
            base_action = match.group(1).strip()
            digit = match.group(2).strip()
            language_code = match.group(3).strip()
        else:
            # the re did not match. let's see if we can match anything from the manual labels
            comment_lower = comment.lower()
            matches = df_manual_labels.loc[df_manual_labels['match'].apply(lambda x: x in comment_lower)] # look for words
            if len(matches) > 0:
                base_action = matches['label'].value_counts().idxmax() # take the most common label from the matches
            else:
                base_action = ""
            digit = ""
            language_code = ""
    return {
        "action_base": base_action,
        "action_digit": digit,
        "action_language": language_code
    }

In [ ]:
df = pd.concat([df, pd.DataFrame.from_records(df['comment'].progress_apply(split_comment))], axis=1)

In [ ]:
df.head()

In [ ]:
df.loc[df["action_base"] == ""]